# Gesture Recognition Assignment 

## By Tushar Prasad

### Problem Statement

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

Thumbs up:  Increase the volume
Thumbs down: Decrease the volume
Left swipe: 'Jump' backwards 10 seconds
Right swipe: 'Jump' forward 10 seconds  
Stop: Pause the movie
 

Each video is a sequence of 30 frames (or images).

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import time

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size,img_idx,width,height):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = img_idx
    x = len(img_idx)    #create a list of image numbers you want to use for a particular video
    y = width
    z = height
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(y,z)).astype(np.float32)
                    else:
                        image = resize(image,(y,z)).astype(np.float32)
                        
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0] / 255    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] / 255    #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] / 255    #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(y,z)).astype(np.float32)
                    else:
                        image = resize(image,(y,z)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = image[:,:,0] / 255    #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] / 255    #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] / 255    #normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)


# training sequences = 663
# validation sequences = 100


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [1]:
from sys import getsizeof
print("Memory util is {} Gigs". format(getsizeof(np.zeros((110,663,30,120,120)))/(1024*1024*1024)))

## for width = 120, height=120, batch size = number of training sequences 
## all 30 images in one sequence, for batch size of 663 i.e all the images
## Using these dimensions in our model, we got Out of Memory error hence, we needed to balance the batch size and image dimensions

NameError: name 'np' is not defined

In [9]:
#Select the frames in sequential order.
img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]

y = 100 #Experiment with image size
z = 100 #Experiment with image size
batch_size = 32 #Experiment with the batch size

num_epochs = 30 #choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 30


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

# Experiment 1 --> Using Conv3D Architecture

## Model 1 - Create a simple base model and fit the data.

In [10]:
#create Conv3d model
model_vanilla = Sequential()
model_vanilla.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_vanilla.add(Activation('relu'))
model_vanilla.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_vanilla.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_vanilla.add(Activation('relu'))
model_vanilla.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model_vanilla.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model_vanilla.add(Activation('relu'))
model_vanilla.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model_vanilla.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model_vanilla.add(Activation('relu'))
model_vanilla.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_vanilla.add(Flatten())
model_vanilla.add(Dropout(0.5))
model_vanilla.add(Dense(256, activation='relu'))
model_vanilla.add(Dropout(0.5))
model_vanilla.add(Dense(5, activation='softmax'))

In [11]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_vanilla.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_vanilla.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 100, 100, 64)  5248      
_________________________________________________________________
activation (Activation)      (None, 18, 100, 100, 64)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 50, 100, 64)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 50, 100, 128)   221312    
_________________________________________________________________
activation_1 (Activation)    (None, 9, 50, 100, 128)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 25, 50, 128)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 4, 25, 50, 256)    8

In [12]:
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

In [13]:
model_name = 'model_vanila' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
start = time.time()

hist = model_vanilla.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 1.6154 - categorical_accuracy: 0.1931Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_vanila_2021-08-3119_45_15.988566/model-00001-1.61539-0.19306-1.60409-0.19000.h5
21/21 [==============================] - 138s 7s/step - loss: 1.6154 - categorical_accuracy: 0.1931 - val_loss: 1.6041 - val_categorical_accuracy: 0.1900
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 1.6052 - categorical_accuracy: 0.2172
Epoch 00002: saving model to model_vanila_2021-08-3119_45_15.988566/model-00002-1.60518-0.21719-1.60008-0.23000.h5
21/21 [==============================] - 134s 6s/step - loss: 1.6052 - categorical_accuracy: 0.2172 - val_loss: 1.6001 - val_categorical_accuracy: 0.2300
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 1.6051 - categorical_accuracy: 0.2142
Epoch 00003:

## Model 2 - Adding batch normalization to conv layers

In [16]:
model_normalized= Sequential()
model_normalized.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_normalized.add(Activation('relu'))
model_normalized.add(BatchNormalization())
model_normalized.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_normalized.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_normalized.add(Activation('relu'))
model_normalized.add(BatchNormalization())
model_normalized.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model_normalized.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model_normalized.add(Activation('relu'))
model_normalized.add(BatchNormalization())
model_normalized.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model_normalized.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model_normalized.add(Activation('relu'))
model_normalized.add(BatchNormalization())
model_normalized.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_normalized.add(Flatten())
model_normalized.add(Dropout(0.5))
model_normalized.add(Dense(128, activation='relu'))
model_normalized.add(Dropout(0.5))
model_normalized.add(Dense(5, activation='softmax'))

In [17]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_normalized.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_normalized.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 18, 100, 100, 64)  5248      
_________________________________________________________________
activation_4 (Activation)    (None, 18, 100, 100, 64)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 100, 100, 64)  256       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 9, 50, 100, 64)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 9, 50, 100, 128)   221312    
_________________________________________________________________
activation_5 (Activation)    (None, 9, 50, 100, 128)   0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 50, 100, 128)  

In [18]:
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

model_name = 'model_normalized' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [19]:
start = time.time()

hist = model_normalized.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 2.5414 - categorical_accuracy: 0.2866Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_normalized_2021-08-3119_45_15.988566/model-00001-2.54139-0.28658-1.54229-0.33000.h5
21/21 [==============================] - 89s 4s/step - loss: 2.5414 - categorical_accuracy: 0.2866 - val_loss: 1.5423 - val_categorical_accuracy: 0.3300
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 1.3931 - categorical_accuracy: 0.4193
Epoch 00002: saving model to model_normalized_2021-08-3119_45_15.988566/model-00002-1.39312-0.41931-2.00196-0.25000.h5
21/21 [==============================] - 90s 4s/step - loss: 1.3931 - categorical_accuracy: 0.4193 - val_loss: 2.0020 - val_categorical_accuracy: 0.2500
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 1.2187 - categorical_accuracy: 0.4766
Epoch 

## Model 3 - Adding dropout layer to conv. layers

In [20]:
model_dropout = Sequential()
model_dropout.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_dropout.add(Activation('relu'))
model_dropout.add(BatchNormalization())
model_dropout.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_dropout.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_dropout.add(Activation('relu'))
model_dropout.add(BatchNormalization())
model_dropout.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_dropout.add(Dropout(0.25))

model_dropout.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model_dropout.add(Activation('relu'))
model_dropout.add(BatchNormalization())
model_dropout.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_dropout.add(Dropout(0.25))

model_dropout.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model_dropout.add(Activation('relu'))
model_dropout.add(BatchNormalization())
model_dropout.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_dropout.add(Flatten())
model_dropout.add(Dropout(0.25))
model_dropout.add(Dense(128, activation='relu'))
model_dropout.add(Dropout(0.25))
model_dropout.add(Dense(5, activation='softmax'))

In [21]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_dropout.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_dropout.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 18, 100, 100, 64)  5248      
_________________________________________________________________
activation_8 (Activation)    (None, 18, 100, 100, 64)  0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 18, 100, 100, 64)  256       
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 9, 50, 100, 64)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 9, 50, 100, 128)   221312    
_________________________________________________________________
activation_9 (Activation)    (None, 9, 50, 100, 128)   0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 9, 50, 100, 128)  

In [22]:
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

model_name = 'model_dropout' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
start = time.time()

hist = model_dropout.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 2.3515 - categorical_accuracy: 0.3047Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_dropout_2021-08-3119_45_15.988566/model-00001-2.35148-0.30468-1.74753-0.19000.h5
21/21 [==============================] - 94s 4s/step - loss: 2.3515 - categorical_accuracy: 0.3047 - val_loss: 1.7475 - val_categorical_accuracy: 0.1900
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 1.2898 - categorical_accuracy: 0.4374
Epoch 00002: saving model to model_dropout_2021-08-3119_45_15.988566/model-00002-1.28982-0.43741-2.38741-0.22000.h5
21/21 [==============================] - 95s 5s/step - loss: 1.2898 - categorical_accuracy: 0.4374 - val_loss: 2.3874 - val_categorical_accuracy: 0.2200
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 1.1070 - categorical_accuracy: 0.5505
Epoch 00003:

## Model 4 - Experiment with filter size(choosing smaller filter)

In [24]:
model_filter1 = Sequential()
model_filter1.add(Conv3D(64, (2,2,2), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_filter1.add(Activation('relu'))
model_filter1.add(BatchNormalization())
model_filter1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_filter1.add(Conv3D(128, (2,2,2), strides=(1,1,1), padding='same'))
model_filter1.add(Activation('relu'))
model_filter1.add(BatchNormalization())
model_filter1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_filter1.add(Dropout(0.25))

model_filter1.add(Conv3D(256, (2,2,2), strides=(1,1,1), padding='same'))
model_filter1.add(Activation('relu'))
model_filter1.add(BatchNormalization())
model_filter1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_filter1.add(Dropout(0.25))

model_filter1.add(Conv3D(256, (2,2,2), strides=(1,1,1), padding='same'))
model_filter1.add(Activation('relu'))
model_filter1.add(BatchNormalization())
model_filter1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_filter1.add(Flatten())
model_filter1.add(Dropout(0.25))
model_filter1.add(Dense(128, activation='relu'))
model_filter1.add(Dropout(0.25))
model_filter1.add(Dense(5, activation='softmax'))

In [25]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_filter1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_filter1.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_12 (Conv3D)           (None, 18, 100, 100, 64)  1600      
_________________________________________________________________
activation_12 (Activation)   (None, 18, 100, 100, 64)  0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 18, 100, 100, 64)  256       
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 9, 50, 100, 64)    0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 9, 50, 100, 128)   65664     
_________________________________________________________________
activation_13 (Activation)   (None, 9, 50, 100, 128)   0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 9, 50, 100, 128)  

In [26]:
batch_size = 32
y = 100
z = 100
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

model_name = 'model_filter1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [27]:
start = time.time()

hist = model_filter1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 2.9040 - categorical_accuracy: 0.2760Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_filter1_2021-08-3119_45_15.988566/model-00001-2.90403-0.27602-1.66840-0.16000.h5
21/21 [==============================] - 86s 4s/step - loss: 2.9040 - categorical_accuracy: 0.2760 - val_loss: 1.6684 - val_categorical_accuracy: 0.1600
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 1.2906 - categorical_accuracy: 0.4284
Epoch 00002: saving model to model_filter1_2021-08-3119_45_15.988566/model-00002-1.29057-0.42836-2.08099-0.10000.h5
21/21 [==============================] - 86s 4s/step - loss: 1.2906 - categorical_accuracy: 0.4284 - val_loss: 2.0810 - val_categorical_accuracy: 0.1000
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 1.1266 - categorical_accuracy: 0.5400
Epoch 00003:

## Model 5 - Model with 5 convolution layers

In [28]:
model_complex1 = Sequential()
model_complex1.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_complex1.add(Activation('relu'))
model_complex1.add(BatchNormalization())
model_complex1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

model_complex1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model_complex1.add(Activation('relu'))
model_complex1.add(BatchNormalization())

model_complex1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model_complex1.add(Activation('relu'))
model_complex1.add(BatchNormalization())
model_complex1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_complex1.add(Dropout(0.25))

model_complex1.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_complex1.add(Activation('relu'))
model_complex1.add(BatchNormalization())
model_complex1.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_complex1.add(Activation('relu'))
model_complex1.add(BatchNormalization())
model_complex1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_complex1.add(Dropout(0.25))

model_complex1.add(Flatten())
model_complex1.add(Dropout(0.25))
model_complex1.add(Dense(64, activation='relu'))
model_complex1.add(Dropout(0.25))

model_complex1.add(Dense(5, activation='softmax'))

In [29]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_complex1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_complex1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 100, 100, 32)  2624      
_________________________________________________________________
activation (Activation)      (None, 18, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 50, 99, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 50, 99, 64)     55360     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 50, 99, 64)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 50, 99, 64)     2

In [30]:
batch_size = 32
y = 100
z = 100
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

model_name = 'model_complex1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [31]:
start = time.time()

hist = model_complex1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 2.4423 - categorical_accuracy: 0.2383Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_complex1_2021-09-0103_05_02.982625/model-00001-2.44228-0.23831-2.67443-0.16000.h5
21/21 [==============================] - 86s 4s/step - loss: 2.4423 - categorical_accuracy: 0.2383 - val_loss: 2.6744 - val_categorical_accuracy: 0.1600
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 1.5315 - categorical_accuracy: 0.3183
Epoch 00002: saving model to model_complex1_2021-09-0103_05_02.982625/model-00002-1.53150-0.31825-4.36950-0.17000.h5
21/21 [==============================] - 88s 4s/step - loss: 1.5315 - categorical_accuracy: 0.3183 - val_loss: 4.3695 - val_categorical_accuracy: 0.1700
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 1.5162 - categorical_accuracy: 0.3092
Epoch 0000

- Adding extra layer did not help to imporve the learning. We do not see any marginal differnce.

## Model 6 - Creating a simpler model with reduced tainable parameters

In [32]:
model_simple1= Sequential()
model_simple1.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_simple1.add(Activation('relu'))
model_simple1.add(BatchNormalization())
model_simple1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

model_simple1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model_simple1.add(Activation('relu'))
model_simple1.add(BatchNormalization())
model_simple1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_simple1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model_simple1.add(Activation('relu'))
model_simple1.add(BatchNormalization())
model_simple1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_simple1.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_simple1.add(Activation('relu'))
model_simple1.add(BatchNormalization())
model_simple1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_simple1.add(Flatten())
model_simple1.add(Dropout(0.25))
model_simple1.add(Dense(128, activation='relu'))
model_simple1.add(Dropout(0.25))

model_simple1.add(Dense(5, activation='softmax'))

In [33]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_simple1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_simple1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 18, 100, 100, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 18, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 18, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 50, 99, 32)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 9, 50, 99, 64)     55360     
_________________________________________________________________
activation_6 (Activation)    (None, 9, 50, 99, 64)     0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 9, 50, 99, 64)    

In [34]:
batch_size = 32
y = 100
z = 100
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

model_name = 'model_simple1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [35]:
start = time.time()

hist = model_simple1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 1.9771 - categorical_accuracy: 0.3891Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_simple1_2021-09-0103_05_02.982625/model-00001-1.97714-0.38914-1.61279-0.22000.h5
21/21 [==============================] - 86s 4s/step - loss: 1.9771 - categorical_accuracy: 0.3891 - val_loss: 1.6128 - val_categorical_accuracy: 0.2200
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 0.9597 - categorical_accuracy: 0.6063
Epoch 00002: saving model to model_simple1_2021-09-0103_05_02.982625/model-00002-0.95974-0.60633-1.91952-0.17000.h5
21/21 [==============================] - 87s 4s/step - loss: 0.9597 - categorical_accuracy: 0.6063 - val_loss: 1.9195 - val_categorical_accuracy: 0.1700
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 0.7377 - categorical_accuracy: 0.7089
Epoch 00003:

### Model 6 is the best model which we could find.

## Model 7 - Reducing more parameters and simpler model improved results

In [11]:
model_reduced = Sequential()
model_reduced.add(Conv3D(16, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,100,100,3)))
model_reduced.add(Activation('relu'))
model_reduced.add(BatchNormalization())
model_reduced.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_reduced.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same'))
model_reduced.add(Activation('relu'))
model_reduced.add(BatchNormalization())
model_reduced.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_reduced.add(Dropout(0.25))

model_reduced.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model_reduced.add(Activation('relu'))
model_reduced.add(BatchNormalization())
model_reduced.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_reduced.add(Dropout(0.25))

model_reduced.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model_reduced.add(Activation('relu'))
model_reduced.add(BatchNormalization())
model_reduced.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_reduced.add(Flatten())
model_reduced.add(Dense(64, activation='relu'))
model_reduced.add(BatchNormalization())
model_reduced.add(Dropout(0.25))

model_reduced.add(Dense(5, activation='softmax'))

In [12]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)    #write your optimizer
model_reduced.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_reduced.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
activation (Activation)      (None, 18, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 50, 50, 32)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 50, 50, 32)     1

In [13]:
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)

model_name = 'model_reduced' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [39]:
start = time.time()

hist = model_reduced.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 1.7977 - categorical_accuracy: 0.3288Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_reduced_2021-09-0103_05_02.982625/model-00001-1.79770-0.32881-1.64155-0.28000.h5
21/21 [==============================] - 87s 4s/step - loss: 1.7977 - categorical_accuracy: 0.3288 - val_loss: 1.6415 - val_categorical_accuracy: 0.2800
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 1.2963 - categorical_accuracy: 0.4827
Epoch 00002: saving model to model_reduced_2021-09-0103_05_02.982625/model-00002-1.29629-0.48265-1.80606-0.22000.h5
21/21 [==============================] - 86s 4s/step - loss: 1.2963 - categorical_accuracy: 0.4827 - val_loss: 1.8061 - val_categorical_accuracy: 0.2200
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 0.9543 - categorical_accuracy: 0.6109
Epoch 00003:

### Running model 7 for 100 epochs to see if it will beat model model 6

In [14]:
num_epochs = 100
start = time.time()

hist = model_reduced.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                         callbacks=callbacks_list, validation_data=val_generator,
                         validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/100
21/21 [==============================] - ETA: 0s - loss: 1.7747 - categorical_accuracy: 0.3529Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_reduced_2021-09-0110_37_03.469651/model-00001-1.77470-0.35294-1.80958-0.21000.h5
21/21 [==============================] - 96s 5s/step - loss: 1.7747 - categorical_accuracy: 0.3529 - val_loss: 1.8096 - val_categorical_accuracy: 0.2100
Epoch 2/100
21/21 [==============================] - ETA: 0s - loss: 1.2548 - categorical_accuracy: 0.5309
Epoch 00002: saving model to model_reduced_2021-09-0110_37_03.469651/model-00002-1.25479-0.53092-2.34330-0.22000.h5
21/21 [==============================] - 91s 4s/step - loss: 1.2548 - categorical_accuracy: 0.5309 - val_loss: 2.3433 - val_categorical_accuracy: 0.2200
Epoch 3/100
21/21 [==============================] - ETA: 0s - loss: 1.0672 - categorical_accuracy: 0.5701
Epoch 000

#### Model 7 didnt beat model 6 which only ran for 30 epochs

# Experiment 2 Model 8 --> Using CNN + LSTM Architecture

#### Trying to run the CNN + LSTM model for a batch size of 100 and 100 epochs to see if it will beat model 6

In [10]:
img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
y = 100
z = 100
batch_size = 100 #experiment with the batch size

num_epochs = 100# choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 100


In [12]:
#write your model here
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional
from tensorflow.keras import optimizers

input_shape=(len(img_idx),y,z,3)

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                          input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu'),
                          input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(128))
model.add(Dropout(0.25))


model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [13]:
## Using Adam Optimizer

optimiser = optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 100, 100, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 100, 100, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 50, 50, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 50, 50, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 50, 50, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 25, 25, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 25, 25, 64)    1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)


In [16]:
model_name = 'model_trail_16' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
start = time.time()

hist = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 100
Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 1.5521 - categorical_accuracy: 0.2941Source path =  /home/datasets/Project_data/val ; batch size = 100

Epoch 00001: saving model to model_trail_16_2021-09-0110_41_01.664535/model-00001-1.55212-0.29412-1.59556-0.32000.h5
7/7 [==============================] - 88s 13s/step - loss: 1.5521 - categorical_accuracy: 0.2941 - val_loss: 1.5956 - val_categorical_accuracy: 0.3200
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 1.2453 - categorical_accuracy: 0.5234
Epoch 00002: saving model to model_trail_16_2021-09-0110_41_01.664535/model-00002-1.24532-0.52338-1.63466-0.23000.h5
7/7 [==============================] - 90s 13s/step - loss: 1.2453 - categorical_accuracy: 0.5234 - val_loss: 1.6347 - val_categorical_accuracy: 0.2300
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 1.0653 - categorical_accuracy: 0.6018
Epoch 00003: 

## Model 9 - USING CNN + GRU Architecture

#### Running the same model with GRU instead of LSTM and lower image dimensions and batch size

In [10]:
img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
y = 80
z = 80
batch_size = 64 #experiment with the batch size

num_epochs = 75# choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 75


In [12]:
#write your model here
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional
from tensorflow.keras import optimizers

input_shape=(len(img_idx),y,z,3)

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                          input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu'),
                          input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(GRU(128))
model.add(Dropout(0.25))


model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [13]:
## Again using Adam optimizer
optimiser = optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 80, 80, 16)    448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 80, 80, 16)    64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 40, 40, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 40, 40, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 40, 40, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 20, 20, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 20, 20, 64)    1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator = generator(train_path, train_doc, batch_size,img_idx,y,z)
val_generator = generator(val_path, val_doc, batch_size,img_idx,y,z)


In [16]:
model_name = 'model_trail_17' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
start = time.time()

hist = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end = time.time()
print('Time taken ', end-start,' seconds')

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/75
11/11 [==============================] - ETA: 0s - loss: 1.5710 - categorical_accuracy: 0.3454Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: saving model to model_trail_17_2021-09-0114_25_44.724385/model-00001-1.57096-0.34540-1.78594-0.23000.h5
11/11 [==============================] - 85s 8s/step - loss: 1.5710 - categorical_accuracy: 0.3454 - val_loss: 1.7859 - val_categorical_accuracy: 0.2300
Epoch 2/75
11/11 [==============================] - ETA: 0s - loss: 1.1458 - categorical_accuracy: 0.5309
Epoch 00002: saving model to model_trail_17_2021-09-0114_25_44.724385/model-00002-1.14578-0.53092-2.21783-0.19000.h5
11/11 [==============================] - 84s 8s/step - loss: 1.1458 - categorical_accuracy: 0.5309 - val_loss: 2.2178 - val_categorical_accuracy: 0.1900
Epoch 3/75
11/11 [==============================] - ETA: 0s - loss: 0.8968 - categorical_accuracy: 0.6772
Epoch 0000

### CNN + LSTM has better accuracy than CNN + GRU but takes more time per epoch

### Yet none of them perform better than Model 6 

## Results:

We used various techiques to find the best model for this problem :-
    1. Changing layers.
    2. Changing strides.
    3. Batch size.
    4. Image Dimensions.
    5. Optimizers
    6. Model Approaches (Conv3D and CNN-RNN)
    7. Frame Sizes
    
Across all these experiments we found out that model 6 is the best model according to the Categorical Train Accuracy, Categorical Validation Accuracy and has a reasonable fast time as written in the write up.